In [1]:
import torch

print(torch.cuda.is_available())

True


In [6]:
import pandas as pd

data = pd.read_csv('./data/full_train_1.csv')
data_final = data[['Comment', 'Rating']]
data_final.to_csv('./data/train_data.csv', index=False)

In [8]:
from loader.dataset import SentimentDataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=True)

trainset = SentimentDataset('./data/train_data.csv',tokenizer = tokenizer)
train_loader = DataLoader(trainset, batch_size = 8, shuffle=True)




Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/loi/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/a47f76f1ac0b6b89f82d15dbe9829935a0014c38/config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "PhobertTokenizer",
  "transformers_version": "4.30.2",
  "type_vocab_size": 1,
  "use_cache": true,
 

In [1]:
import py_vncorenlp
#py_vncorenlp.download_model(save_dir='/home/loi/loi_code/sentiment_analysis/vncorenlp')

In [2]:
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/home/loi/loi_code/sentiment_analysis/vncorenlp')
text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

output = rdrsegmenter.word_segment(text)

print(output)

2023-07-23 15:19:08 INFO  WordSegmenter:24 - Loading Word Segmentation model
['Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội .', 'Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .']


In [5]:
print(type(output[0]))

<class 'str'>


In [6]:
text = "".join(e for e in output)
text

'Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội .Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .'

In [13]:
cd ..

/home/loi/loi_code/sentiment_analysis


In [7]:
from gensim.utils import simple_preprocess

def get_input_data(text):
    #preprocessing: {remove icon, special charater, lower}
    text = ' '.join(simple_preprocess(text))

    return text

In [8]:
text_process = get_input_data(text)
text_process

'ông đang làm_việc tại đại_học quốc_gia hà_nội bà lan vợ ông chúc cũng làm_việc tại đây'

In [12]:
!pwd

/home/loi/loi_code/sentiment_analysis/vncorenlp


In [15]:
import pandas as pd

data = pd.read_csv('./data/data_all.csv')
data.drop(['index'], axis=1 , inplace=True)

In [17]:
def words_segment(text):
    output = rdrsegmenter.word_segment(text)
    return "".join(e for e in output)

In [24]:
data['content'] = data['content'].apply(words_segment)
data

,content,label
0,đặt_hàng hôm_nay tới mới có vậy,0
1,rất tốt cho luôn chỉ tiếc là không có sao ngôi...,1
2,toàn quảng_cáo rác,0
3,không có tìm_kiếm bằng hình_ảnh mặt không có m...,0
4,mặt cười với đôi mắt mỉm cười mặt đỏ_ửng tàn m...,1
...,...,...
373001,cước vẫn chuyển cao,1
373002,lazada mua sam tren mang tien that nhi dụng ho...,1
373003,sao mua không dịch_vụ,1
373004,mua hàng mà không gửi thông_tin về gmail vậy,1


In [21]:
data['content'] = data['content'].apply(str)

In [23]:
data

,content,label
0,đặt hàng hôm nay tới mới có vậy,0
1,rất tốt cho luôn chỉ tiếc là không có sao ng...,1
2,toàn quảng cáo rác,0
3,không có tìm kiếm bằng hình ảnh mặt không có m...,0
4,mặt cười với đôi mắt mỉm cười mặt đỏ ửng tàn m...,1
...,...,...
373001,cước vẫn chuyển cao,1
373002,lazada mua sam tren mang tien that nhi dụng ho...,1
373003,sao mua không dịch vụ,1
373004,mua hàng mà không gửi thông tin về gmail vậy,1


In [25]:
data.to_csv('data/data_segment.csv', index=False)